In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
## load baseline simulated data
simData = pd.read_csv('../DSSG2023-Heating-Loads-Data/heatPumpCal/results_BoroughProjs.csv')
simData['Census_Area_LowerCase'] = simData['Census_Area'].str.lower()

In [3]:
simData.head()

,Unnamed: 0,City,Exist_Fuel_Type,Scenario,Rebate_dol,Fuel_Esc_Rate,Temp_Projection,Census_Area,ANCSA_Region,Util_Name,...,CO2_lbs_saved,CO2_driving_miles_saved,Fuel_Use_Chg,Fuel_Price_Incremental,Elec_Use_Chg,Elec_Rate_Incremental,Elec_Rate_Avg_Base,Econ,Exist_Fuel,Census_Area_LowerCase
0,0,Adak,1,Base,0,0.03,0.0,Aleutians West Census Area,Aleut,Adak -TDX Residential,...,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,1.051414,Economic,Electricity,aleutians west census area
1,1,Adak,1,Base,0,0.06,0.0,Aleutians West Census Area,Aleut,Adak -TDX Residential,...,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,1.051414,Economic,Electricity,aleutians west census area
2,2,Adak,1,Base,0,0.09,0.0,Aleutians West Census Area,Aleut,Adak -TDX Residential,...,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,1.051414,Economic,Electricity,aleutians west census area
3,3,Adak,1,Base,2000,0.03,0.0,Aleutians West Census Area,Aleut,Adak -TDX Residential,...,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,1.051414,Economic,Electricity,aleutians west census area
4,4,Adak,1,Base,2000,0.06,0.0,Aleutians West Census Area,Aleut,Adak -TDX Residential,...,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,1.051414,Economic,Electricity,aleutians west census area


In [4]:
## loading heat fule type data
heatData = pd.read_excel('../DSSG2023-Heating-Loads-Data/Borough Heating Prop.xlsx')

In [5]:
heatData.head()

,Census_Area,Natural gas,Propane,Electricity,Oil
0,Aleutians East Borough,0.003623,0.009662,0.021739,0.964976
1,Aleutians West Census Area,0.012357,0.006178,0.060900,0.920565
2,Anchorage municipality,0.804394,0.010960,0.157433,0.027214
3,Bethel Census Area,0.002978,0.009680,0.020849,0.966493
4,Bristol Bay Borough,0.019293,0.000000,0.003215,0.977492


In [6]:
## loading places data
places = gpd.read_file('../DSSG2023-Heating-Loads-Data/shapefiles/Places2010/Places.shp')

In [7]:
## selecting specific columns
places = places[['NAME', 'HOUSEUNITS']]

In [8]:
places.head()

,NAME,HOUSEUNITS
0,Sand Point city,290.0
1,King Cove city,229.0
2,False Pass city,40.0
3,Cold Bay city,82.0
4,Akutan city,44.0


In [10]:
## load Alan's city csv
city_meta = pd.read_csv('../DSSG2023-Heating-Loads-Data/heatPumpCal/city.csv')

In [11]:
city_meta = city_meta[['Name', 'census_city', 'census_area']]

In [12]:
city_meta.head()

,Name,census_city,census_area
0,Anchorage,Anchorage municipality,Anchorage municipality
1,Adak,Adak city,Aleutians West Census Area
2,Akutan,Akutan city,Aleutians East Borough
3,Allakaket,Allakaket city,Yukon-Koyukuk Census Area
4,Angoon,Angoon city,Hoonah-Angoon Census Area


## Find the place (city)'s house unit, and calculate the weighted average of outcomes by houseunit in a borough

In [13]:
## rename places shapefile
places.rename(columns={'NAME':'census_city'}, inplace=True)

In [14]:
## make sure all Alan's cities can be matched perfectly with census cities
city_meta['census_city'].isin(places['census_city']).mean()

1.0

In [15]:
## merge houseunits data to Alan's cities
city_meta_wHU = city_meta.merge(places, on='census_city')

In [16]:
city_meta_wHU.rename(columns={'Name':'City'}, inplace=True)

In [17]:
## merge houseunit data with simulated data
simData_wHU = simData.merge(city_meta_wHU, on='City')

In [18]:
## notice the discrepancy between Alan's cities (277) and census cities (255)
print([len(simData_wHU['census_city'].unique()), len(simData_wHU['City'].unique())])


[255, 277]


In [19]:
## average out across unique census cities
simData_uniqueCities = simData_wHU.groupby(['census_city', 'census_area', 'Exist_Fuel', 'Rebate_dol', 'Fuel_Esc_Rate', 'Temp_Projection'])[['HOUSEUNITS', "NPV", "CO2_lbs_saved", "CO2_driving_miles_saved", "Heating_Days_Above5", "Heating_Days_Below5"]].mean()

In [20]:
simData_uniqueCities.reset_index(inplace=True)

In [21]:
## get unique city-borough dataframe with houseunits
census_city_area_wHU = city_meta_wHU.drop(['City'], axis=1).drop_duplicates()

In [22]:
census_city_area_wHU.sort_values('census_area').head()

,census_city,census_area,HOUSEUNITS
148,Nelson Lagoon CDP,Aleutians East Borough,32.0
7,Akutan city,Aleutians East Borough,44.0
146,False Pass city,Aleutians East Borough,40.0
147,King Cove city,Aleutians East Borough,229.0
149,Sand Point city,Aleutians East Borough,290.0


In [24]:
## find the houseunit proportion (out of a borough) for a given city
df = census_city_area_wHU
HU_prop_df = pd.DataFrame()
for u in df['census_area'].unique():
    sub_df = df[df.census_area== u]
    sub_list = []
    for i in range(0,len(sub_df)):
        sub_list.append(sub_df.iloc[i]["HOUSEUNITS"]/sum(sub_df["HOUSEUNITS"]))
    sub_df2 = sub_df.copy()
    sub_df2["HU_prop"] = sub_list
    HU_prop_df = pd.concat([HU_prop_df,sub_df2])

In [25]:
HU_prop_df

,census_city,census_area,HOUSEUNITS,HU_prop
0,Anchorage municipality,Anchorage municipality,113032.0,1.000000
6,Adak city,Aleutians West Census Area,500.0,0.260010
12,Atka city,Aleutians West Census Area,43.0,0.022361
150,Nikolski CDP,Aleutians West Census Area,23.0,0.011960
151,St. George city,Aleutians West Census Area,61.0,0.031721
...,...,...,...,...
264,Port Lions city,Kodiak Island Borough,113.0,0.042966
265,Fairbanks city,Fairbanks North Star Borough,13056.0,0.827062
272,North Pole city,Fairbanks North Star Borough,916.0,0.058026
273,Salcha CDP,Fairbanks North Star Borough,585.0,0.037058


In [26]:
HU_prop_df.sort_values('census_area').to_csv('../Output/HU_prop_df.csv')

In [27]:
## merge the proportion w/ simulated data
simData_uniqueCities_wHUprop = simData_uniqueCities.merge(HU_prop_df.drop(['census_area'], axis=1), on ='census_city')

## Getting borough-level estimates weighted by city-houseunit-proportion

In [28]:
simData_uniqueCities_wHUprop['NPV_weighted'] = simData_uniqueCities_wHUprop['NPV'] * simData_uniqueCities_wHUprop['HU_prop']
simData_uniqueCities_wHUprop['CO2_lbs_saved_weighted'] = simData_uniqueCities_wHUprop['CO2_lbs_saved'] * simData_uniqueCities_wHUprop['HU_prop']
simData_uniqueCities_wHUprop['CO2_driving_miles_saved_weighted'] = simData_uniqueCities_wHUprop['CO2_driving_miles_saved'] * simData_uniqueCities_wHUprop['HU_prop']
simData_uniqueCities_wHUprop['Heating_Days_Above5_weighted'] = simData_uniqueCities_wHUprop['Heating_Days_Above5'] * simData_uniqueCities_wHUprop['HU_prop']
simData_uniqueCities_wHUprop['Heating_Days_Below5_weighted'] = simData_uniqueCities_wHUprop['Heating_Days_Below5'] * simData_uniqueCities_wHUprop['HU_prop']

In [29]:
borough_estimates_weightedByHU = simData_uniqueCities_wHUprop.groupby(['census_area', 'Exist_Fuel', 'Rebate_dol', 'Fuel_Esc_Rate', 'Temp_Projection'])[["NPV_weighted", "CO2_lbs_saved_weighted", "CO2_driving_miles_saved_weighted", "Heating_Days_Above5_weighted", "Heating_Days_Below5_weighted"]].sum()

In [30]:
borough_estimates_weightedByHU.reset_index(inplace=True)

In [31]:
borough_estimates_weightedByHU

,census_area,Exist_Fuel,Rebate_dol,Fuel_Esc_Rate,Temp_Projection,NPV_weighted,CO2_lbs_saved_weighted,CO2_driving_miles_saved_weighted,Heating_Days_Above5_weighted,Heating_Days_Below5_weighted
0,Aleutians East Borough,#1 Oil,0,0.03,0.0,-1114.817367,1999.199708,2241.584480,365.000000,0.000000
1,Aleutians East Borough,#1 Oil,0,0.03,1.5,-961.837917,1948.211122,2184.413991,364.938633,0.000000
2,Aleutians East Borough,#1 Oil,0,0.03,3.0,-887.967795,1877.801001,2105.467283,364.938633,0.000000
3,Aleutians East Borough,#1 Oil,0,0.06,0.0,-201.489759,1999.199708,2241.584480,365.000000,0.000000
4,Aleutians East Borough,#1 Oil,0,0.06,1.5,-17.543064,1948.211122,2184.413991,364.938633,0.000000
...,...,...,...,...,...,...,...,...,...,...
2425,Yukon-Koyukuk Census Area,Propane,4000,0.06,1.5,17313.429016,17.132531,19.209694,206.593131,46.407530
2426,Yukon-Koyukuk Census Area,Propane,4000,0.06,3.0,16996.397392,14.265970,15.995589,207.472919,42.131110
2427,Yukon-Koyukuk Census Area,Propane,4000,0.09,0.0,21615.855622,1.955880,2.193012,204.904557,50.056473
2428,Yukon-Koyukuk Census Area,Propane,4000,0.09,1.5,21181.959975,17.132531,19.209694,206.593131,46.407530


In [32]:
borough_estimates_weightedByHU.Exist_Fuel.unique()

array(['#1 Oil', 'Electricity', 'Propane', 'Natural Gas'], dtype=object)

## Weighting the borough estimates by fuel use distribution

In [33]:
heatData.rename(columns={'Census_Area':'census_area', 'Natural gas':'Natural Gas', 'Oil':'#1 Oil'}, inplace=True)

In [34]:
heatData.head()

,census_area,Natural Gas,Propane,Electricity,#1 Oil
0,Aleutians East Borough,0.003623,0.009662,0.021739,0.964976
1,Aleutians West Census Area,0.012357,0.006178,0.060900,0.920565
2,Anchorage municipality,0.804394,0.010960,0.157433,0.027214
3,Bethel Census Area,0.002978,0.009680,0.020849,0.966493
4,Bristol Bay Borough,0.019293,0.000000,0.003215,0.977492


In [35]:
borough_estimates_weightedByHU.head()

,census_area,Exist_Fuel,Rebate_dol,Fuel_Esc_Rate,Temp_Projection,NPV_weighted,CO2_lbs_saved_weighted,CO2_driving_miles_saved_weighted,Heating_Days_Above5_weighted,Heating_Days_Below5_weighted
0,Aleutians East Borough,#1 Oil,0,0.03,0.0,-1114.817367,1999.199708,2241.584480,365.000000,0.0
1,Aleutians East Borough,#1 Oil,0,0.03,1.5,-961.837917,1948.211122,2184.413991,364.938633,0.0
2,Aleutians East Borough,#1 Oil,0,0.03,3.0,-887.967795,1877.801001,2105.467283,364.938633,0.0
3,Aleutians East Borough,#1 Oil,0,0.06,0.0,-201.489759,1999.199708,2241.584480,365.000000,0.0
4,Aleutians East Borough,#1 Oil,0,0.06,1.5,-17.543064,1948.211122,2184.413991,364.938633,0.0


In [36]:
heatData_long = pd.melt(heatData, id_vars='census_area', value_vars=['Natural Gas', 'Propane','Electricity', '#1 Oil'], var_name='Exist_Fuel', value_name='fuel_prop')

In [37]:
borough_estimates_wFuelProp = borough_estimates_weightedByHU.merge(heatData_long, on=['census_area', 'Exist_Fuel'])

In [38]:
borough_estimates_wFuelProp.head()

,census_area,Exist_Fuel,Rebate_dol,Fuel_Esc_Rate,Temp_Projection,NPV_weighted,CO2_lbs_saved_weighted,CO2_driving_miles_saved_weighted,Heating_Days_Above5_weighted,Heating_Days_Below5_weighted,fuel_prop
0,Aleutians East Borough,#1 Oil,0,0.03,0.0,-1114.817367,1999.199708,2241.584480,365.000000,0.0,0.964976
1,Aleutians East Borough,#1 Oil,0,0.03,1.5,-961.837917,1948.211122,2184.413991,364.938633,0.0,0.964976
2,Aleutians East Borough,#1 Oil,0,0.03,3.0,-887.967795,1877.801001,2105.467283,364.938633,0.0,0.964976
3,Aleutians East Borough,#1 Oil,0,0.06,0.0,-201.489759,1999.199708,2241.584480,365.000000,0.0,0.964976
4,Aleutians East Borough,#1 Oil,0,0.06,1.5,-17.543064,1948.211122,2184.413991,364.938633,0.0,0.964976


In [39]:
borough_estimates_wFuelProp['NPV_weighted2'] = borough_estimates_wFuelProp['NPV_weighted'] * borough_estimates_wFuelProp['fuel_prop']
borough_estimates_wFuelProp['CO2_lbs_saved_weighted2'] = borough_estimates_wFuelProp['CO2_lbs_saved_weighted'] * borough_estimates_wFuelProp['fuel_prop']
borough_estimates_wFuelProp['CO2_driving_miles_saved_weighted2'] = borough_estimates_wFuelProp['CO2_driving_miles_saved_weighted'] * borough_estimates_wFuelProp['fuel_prop']
borough_estimates_wFuelProp['Heating_Days_Above5_weighted2'] = borough_estimates_wFuelProp['Heating_Days_Above5_weighted'] * borough_estimates_wFuelProp['fuel_prop']
borough_estimates_wFuelProp['Heating_Days_Below5_weighted2'] = borough_estimates_wFuelProp['Heating_Days_Below5_weighted'] * borough_estimates_wFuelProp['fuel_prop']

In [40]:
borough_estimates_wFuelProp.columns

Index(['census_area', 'Exist_Fuel', 'Rebate_dol', 'Fuel_Esc_Rate',
       'Temp_Projection', 'NPV_weighted', 'CO2_lbs_saved_weighted',
       'CO2_driving_miles_saved_weighted', 'Heating_Days_Above5_weighted',
       'Heating_Days_Below5_weighted', 'fuel_prop', 'NPV_weighted2',
       'CO2_lbs_saved_weighted2', 'CO2_driving_miles_saved_weighted2',
       'Heating_Days_Above5_weighted2', 'Heating_Days_Below5_weighted2'],
      dtype='object')

In [41]:
borough_estimates_weightedByHuFuel = borough_estimates_wFuelProp.groupby(['census_area', 'Rebate_dol', 'Fuel_Esc_Rate', 'Temp_Projection'])[['NPV_weighted2', 'CO2_lbs_saved_weighted2', 'CO2_driving_miles_saved_weighted2', 'Heating_Days_Above5_weighted2', 'Heating_Days_Below5_weighted2']].sum()

In [42]:
borough_estimates_weightedByHuFuel.reset_index(inplace=True)

In [43]:
borough_estimates_weightedByHuFuel['Heating_Days_Covered'] = borough_estimates_weightedByHuFuel['Heating_Days_Above5_weighted2'] / (borough_estimates_weightedByHuFuel['Heating_Days_Above5_weighted2'] + borough_estimates_weightedByHuFuel['Heating_Days_Below5_weighted2'])

In [44]:
borough_estimates_weightedByHuFuel.to_csv('../Output/borough_estimates_weightedByHuFuel.csv')